In [1]:
from spd.experiments.attention.train_attn import AttnModelConfig, AttnTrainConfig
from spd.utils import set_seed
from spd.experiments.attention.train_attn import run_train
import torch
from spd.data_utils import SkipTrigramDataset, DatasetGeneratedDataLoader
import plotly.express as px
from datetime import datetime
from pathlib import Path
from typing import Any

import fire
import torch
import wandb
import yaml

from spd.configs import Config, TMSTaskConfig
from spd.data_utils import DatasetGeneratedDataLoader, SparseFeatureDataset
from spd.experiments.attention.models import SingleHeadAttentionModel, AttnModelConfig
from spd.log import logger
from spd.plotting import create_toy_model_plot_results
from spd.run_spd import get_common_run_name_suffix, optimize
from spd.utils import get_device, load_config, set_seed
from spd.wandb_utils import init_wandb
from spd.experiments.attention.attention_decomposition import get_run_name, save_target_model_info, main
from spd.utils import calc_kl_divergence_lm
wandb.require("core")

wandb: WARNING `wandb.require('core')` is a no-op as it is now the default behavior.
wandb: WARNING `wandb.require('core')` is a no-op as it is now the default behavior.
wandb: WARNING `wandb.require('core')` is a no-op as it is now the default behavior.


In [ ]:
import os
os.environ["WANDB_ENTITY"] = "kwyn390"
main("/root/spd/spd/experiments/attention/attn_config.yaml")

2025-07-07 17:23:22 - INFO - Using device: cuda
wandb: Currently logged in as: kwyn390 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


2025-07-07 17:23:27 - INFO - wandb_entity='kwyn390' wandb_project='spd-attn' wandb_run_name=None wandb_run_name_prefix='' seed=0 C=32 n_mask_samples=1 n_ci_mlp_neurons=32 target_module_patterns=['attention'] faithfulness_coeff=10.0 recon_coeff=None stochastic_recon_coeff=5.0 recon_layerwise_coeff=None stochastic_recon_layerwise_coeff=5.0 importance_minimality_coeff=0.001 schatten_coeff=None out_recon_coeff=None embedding_recon_coeff=None is_embed_unembed_recon=False pnorm=1.0 output_loss_type='attn' lr=0.003 steps=50000 batch_size=2048 lr_schedule='cosine' lr_exponential_halflife=None lr_warmup_pct=0.0 n_eval_steps=100 image_freq=5000 image_on_first_step=True print_freq=5000 save_freq=None log_ce_losses=False pretrained_model_class='spd.experiments.attention.models.SingleHeadAttentionModel' pretrained_model_path=PosixPath('/root/spd/spd/experiments/attention/toy_out/attention_vocab20_d16_seq8_datarandom_seed0_20250707_170009_662/attention_model.pth') pretrained_model_name_hf=None pretr

Trying to load model from: /root/spd/spd/experiments/attention/toy_out/attention_vocab20_d16_seq8_datarandom_seed0_20250707_170009_662/attention_model.pth
File exists: True
File size: 7277 bytes
First 10 bytes: b'PK\x03\x04\x00\x00\x08\x08\x00\x00'


2025-07-07 17:23:27 - INFO - Target model parameters frozen.


Generated 32 skip trigrams:
  0, 18 -> 11
  14, 4 -> 11
  4, 5 -> 12
  8, 14 -> 3
  4, 4 -> 0
  7, 8 -> 19
  11, 10 -> 12
  9, 9 -> 8
  3, 16 -> 9
  0, 15 -> 13
  3, 15 -> 7
  3, 0 -> 1
  11, 6 -> 8
  16, 12 -> 8
  6, 1 -> 18
  9, 7 -> 2
  9, 8 -> 8
  10, 5 -> 13
  9, 18 -> 4
  13, 14 -> 11
  15, 9 -> 14
  2, 14 -> 6
  16, 2 -> 9
  2, 18 -> 0
  10, 19 -> 6
  16, 4 -> 13
  19, 5 -> 6
  14, 18 -> 1
  7, 9 -> 14
  16, 7 -> 3
  8, 12 -> 2
  16, 9 -> 8

=== SPD DECOMPOSITION DEBUG ===
Target module patterns: ['attention']
Components created: ['attention']
State Dict Keys: ['model.token_embedding.weight', 'model.attention.weight', 'model.attention.OV', 'model.unembed.weight', 'components.attention.A', 'components.attention.B', 'gates.attention.mlp_in', 'gates.attention.in_bias', 'gates.attention.mlp_out', 'gates.attention.out_bias']

Original modules in target model:
  token_embedding: Embedding
  attention: BaseAttentionModule
  unembed: Linear

Components replacing modules:
  attention: At

2025-07-07 17:23:28 - INFO - Base LR scheduler created: cosine
  0% 0/50001 [00:00<?, ?it/s]2025-07-07 17:23:28 - INFO - Step 0: Generating plots...


--- Step 0 ---
LR: 0.003000
Total Loss: 1.3311056
loss/faithfulness: 0.1122860
loss/stochastic_recon: 0.0202063
loss/stochastic_recon_layerwise: 0.0201925
loss/importance_minimality: 6.2515569


  0% 1/50001 [00:37<516:28:48, 37.19s/it]

Saved plot to /root/spd/spd/experiments/attention/TEST_OUT/nmasks1_stochrecon5.00e+00_stochreconlayer5.00e+00_p1.00e+00_impmin1.00e-03_C32_sd0_lr3.00e-03_bs2048_vocab20_d16_seq8_20250707_172327_316/mask_vals_attention_0.png
Saved plot to /root/spd/spd/experiments/attention/TEST_OUT/nmasks1_stochrecon5.00e+00_stochreconlayer5.00e+00_p1.00e+00_impmin1.00e-03_C32_sd0_lr3.00e-03_bs2048_vocab20_d16_seq8_20250707_172327_316/mean_component_activation_counts_0.png


  0% 72/50001 [01:05<5:30:49,  2.52it/s] 

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
config = AttnTrainConfig(
        wandb_entity="kwyn390",
        wandb_project="attn_toy",
        attention_model_config=AttnModelConfig(
            vocab_size=20,
            d_model = 16,
            seq_len=8,
        
        ),

        batch_size=4096,
        steps=1000,
        seed=0,
        lr=5e-3,
        n_trigrams=32,

        lr_schedule="constant",
        data_type="random"
    )


set_seed(config.seed)

run_train(config, device)




wandb: Currently logged in as: kwyn390 (sae_head_superposition) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Generated 32 skip trigrams:
  0, 18 -> 11
  14, 4 -> 11
  4, 5 -> 12
  8, 14 -> 3
  4, 4 -> 0
  7, 8 -> 19
  11, 10 -> 12
  9, 9 -> 8
  3, 16 -> 9
  0, 15 -> 13
  3, 15 -> 7
  3, 0 -> 1
  11, 6 -> 8
  16, 12 -> 8
  6, 1 -> 18
  9, 7 -> 2
  9, 8 -> 8
  10, 5 -> 13
  9, 18 -> 4
  13, 14 -> 11
  15, 9 -> 14
  2, 14 -> 6
  16, 2 -> 9
  2, 18 -> 0
  10, 19 -> 6
  16, 4 -> 13
  19, 5 -> 6
  14, 18 -> 1
  7, 9 -> 14
  16, 7 -> 3
  8, 12 -> 2
  16, 9 -> 8


2025-07-07 17:00:10 - INFO - Saved config to /root/spd/spd/experiments/attention/toy_out/attention_vocab20_d16_seq8_datarandom_seed0_20250707_170009_662/attention_train_config.yaml
  0% 1/1000 [00:01<17:58,  1.08s/it, loss=3, lr=0.005]

token_embedding.weight: grad_norm=0.022402
attention.weight: grad_norm=0.000000
attention.OV: grad_norm=0.000108
unembed.weight: grad_norm=0.023069

Step 0 debugging:
Sequences shape: torch.Size([4096, 8])
Targets shape: torch.Size([4096])
Sample sequence: [ 1 17 11  4 13  4 14 11]
Sample target: 11
Last token in sequence: 11
Logits shape: torch.Size([4096, 8, 20])
Final logits shape: torch.Size([4096, 20])
Loss: 2.995866060256958
Predicted token: 18, Target: 11
Step 0 Loss: 2.995866060256958


 10% 101/1000 [01:20<11:53,  1.26it/s, loss=0.106, lr=0.005]

...

token_embedding.weight: grad_norm=0.000734
attention.weight: grad_norm=0.000229
attention.OV: grad_norm=0.000444
unembed.weight: grad_norm=0.000518

Step 800 debugging:
Sequences shape: torch.Size([4096, 8])
Targets shape: torch.Size([4096])
Sample sequence: [ 4 15  5  3 18  7 16 12]
Sample target: 12
Last token in sequence: 12
Logits shape: torch.Size([4096, 8, 20])
Final logits shape: torch.Size([4096, 20])
Loss: 0.00044313890975899994
Predicted token: 12, Target: 12
Step 800 Loss: 0.00044313890975899994


 90% 901/1000 [11:54<01:18,  1.26it/s, loss=0.000359, lr=0.005]

token_embedding.weight: grad_norm=0.000611
attention.weight: grad_norm=0.000187
attention.OV: grad_norm=0.000373
unembed.weight: grad_norm=0.000430

Step 900 debugging:
Sequences shape: torch.Size([4096, 8])
Targets shape: torch.Size([4096])
Sample sequence: [ 8  3  6  0 14  5 10 13]
Sample target: 13
Last token in sequence: 13
Logits shape: torch.Size([4096, 8, 20])
Final logits shape: torch.Size([4096, 20])
Loss: 0.0003592415596358478
Predicted token: 13, Target: 13
Step 900 Loss: 0.0003592415596358478


100% 1000/1000 [13:12<00:00,  1.26it/s, loss=0.000305, lr=0.005]
2025-07-07 17:13:23 - INFO - Saved model to /root/spd/spd/experiments/attention/toy_out/attention_vocab20_d16_seq8_datarandom_seed0_20250707_170009_662/attention_model.pth


Step 999 Loss: 0.0003052981337532401


In [ ]:
AttnTrainConfig()

In [2]:
import torch
from spd.experiments.attention.models import SingleHeadAttentionModel, AttnModelConfig

# Recreate the exact same model config used during training
model_config = AttnModelConfig(
    vocab_size=20,
    d_model=16,
    seq_len=8,
)

# Create the model
base_model = SingleHeadAttentionModel(model_config)

# Load the saved state dict
model_path = "/root/spd/spd/experiments/attention/TEST_OUT/nmasks1_stochrecon1.00e+01_stochreconlayer1.00e+01_p1.00e+00_impmin1.00e-03_C11_sd0_lr1.00e-03_bs2048_vocab20_d16_seq8_20250707_115748_969/attn.pth"
state_dict = torch.load(model_path)  # Use map_location if loading on different device
base_model.load_state_dict(state_dict)

# Set to evaluation mode for inference

# Move to device if needed
device = "cuda" if torch.cuda.is_available() else "cpu"
base_model.to(device)

SingleHeadAttentionModel(
  (token_embedding): Embedding(20, 16)
  (attention): BaseAttentionModule()
  (unembed): Linear(in_features=16, out_features=20, bias=False)
)

In [3]:
from spd.utils import load_config
from spd.models.component_model import ComponentModel
model_path = "/root/spd/spd/experiments/attention/TEST_OUT/nmasks1_stochrecon5.00e+00_stochreconlayer5.00e+00_p1.00e+00_impmin1.00e-03_C16_sd0_lr3.00e-03_bs2048_vocab20_d16_seq8_20250707_141040_697/model_50000.pth"
cfg_path = "/root/spd/spd/experiments/attention/attn_config.yaml"
config = load_config(cfg_path, config_model=Config)
state_dict = torch.load(model_path)  # Use map_location if loading on different device
state_dict.keys()
model = ComponentModel(
        base_model=base_model,
        target_module_patterns=config.target_module_patterns,
        C=config.C,
        n_ci_mlp_neurons=config.n_ci_mlp_neurons,
        pretrained_model_output_attr=config.pretrained_model_output_attr,
    )
model.load_state_dict(state_dict)

<All keys matched successfully>

In [5]:

trigrams =  [(10, 0, 1), (11, 0, 5), (12, 0, 5), (13, 0, 6), (14, 0, 9), (15, 0, 6), (16, 0, 8), (17, 0, 9), (18, 0, 3), (19, 0, 8)]


In [10]:
import torch
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def analyze_trigram_components(model, trigrams, device="cuda"):
    """Analyze which components activate for each skip trigram."""
    
    # Get token embeddings
    token_embeds = model.model.token_embedding.weight  # [vocab_size, d_model]
    
    # Extract A and B matrices
    A = model.components['attention'].A  # [d_model, C]
    B = model.components['attention'].B  # [C, d_model]
    
    results = []
    
    for i, (first, second, third) in enumerate(trigrams):
        # Get embeddings for the positions we care about
        pos1_embed = token_embeds[[first]][0].cuda() # Query position
        pos2_embed = token_embeds[[second]][0].cuda()# Key position (the skipped one)
        
        # Compute component activations
        # Query activations: pos1_embed @ A
        query_acts = torch.einsum('d,dc->c', pos1_embed.cuda(), A.cuda())  # [C]
        
        # Key activations: pos2_embed @ B.T  
        key_acts = torch.einsum('d,cd->c', pos2_embed, B.cuda())    # [C]
        
        # Component-wise attention scores (before summing)
        component_scores = query_acts * key_acts  # [C]
        
        results.append({
            'trigram_idx': i,
            'trigram': (first, second, third),
            'query_token': first,
            'key_token': second,
            'target_token': third,
            'query_acts': query_acts.cpu().detach().numpy(),
            'key_acts': key_acts.cpu().detach().numpy(), 
            'component_scores': component_scores.cpu().detach().numpy()
        })
    
    return results

# Run the analysis
results = analyze_trigram_components(model, trigrams, device)

In [11]:
def plot_component_heatmap(results):
    """Create a heatmap showing component activations for all trigrams."""
    
    # Extract data
    component_scores_matrix = np.array([r['component_scores'] for r in results])
    trigram_labels = [f"T{i}: {r['trigram']}" for i, r in enumerate(results)]
    component_labels = [f"C{i}" for i in range(len(results[0]['component_scores']))]
    
    # Create heatmap
    fig = go.Figure(data=go.Heatmap(
        z=component_scores_matrix,
        x=component_labels,
        y=trigram_labels,
        colorscale='RdBu',
        zmid=0,
        colorbar=dict(title="Activation Score"),
        hovertemplate="Trigram: %{y}<br>Component: %{x}<br>Score: %{z:.3f}<extra></extra>"
    ))
    
    fig.update_layout(
        title="Component Activations Across All Skip Trigrams",
        xaxis_title="Components",
        yaxis_title="Trigrams",
        height=800,
        width=1000,
        font=dict(size=10)
    )
    
    fig.show()
    return fig

plot_component_heatmap(results)

In [12]:
token_embeds = model.model.token_embedding.weight  # [vocab_size, d_model]
ci_gates = model.gates.attention.cuda()
A = model.components['attention'].A  # [d_model, C]
B = model.components['attention'].B  # [C, d_model]
a = []
for i, (first, second, third) in enumerate(trigrams):
        # Get embeddings for the positions we care about
        pos1_embed = token_embeds[[first]][0].cuda() 
        pos2_embed = token_embeds[[second]][0].cuda()# Key position (the skipped one)
        query_acts = torch.einsum('d,dc->c', pos1_embed.cuda(), A.cuda())  # [C]
 
        key_acts = torch.einsum('d,cd->c', pos2_embed, B.cuda())    # [C]
        component_scores = query_acts * key_acts  
        cis = ci_gates.forward(component_scores.cuda())
        a.append(cis)
        
        
 

In [13]:
px.imshow(torch.stack(a).detach().cpu(), color_continuous_midpoint=0, color_continuous_scale='RdBu')

In [6]:
test = torch.tensor([[ 1,  5,  5,  9 , 5, 1,  4,  0]]).cuda()
components = {k.removeprefix("components.").replace("-", "."): v.cuda() for k, v in model.components.items()}
out = model.forward_with_components(test.cuda(), components = components)
out[0, -1]


tensor([-7.0818,  3.4290, -6.9955,  2.8887, -7.1337,  4.2033, 14.3580, -2.2763,
         0.7044,  5.7110, -7.0152, -6.9950, -7.0617, -7.0288, -6.9360, -6.9916,
        -6.9091, -7.0190, -6.9583, -7.0637], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [5]:
# Use the components directly without key manipulation
test = torch.tensor([[ 1,  5,  5,  13 , 5, 1,  4,  0]]).cuda()
out = model.forward_with_components(test, components=model.components.cuda())
gt = base_model.forward(test)

print(f"Component output: {out[0, -1]}")
print(f"Base model output: {gt[0, -1]}")
print(f"Are they close? {torch.allclose(out, gt, atol=1e-3)}")

Component output: tensor([-7.1031,  3.3072, -6.9840,  2.9586, -7.1021,  4.4650, 14.5908, -2.6494,
         0.9504,  5.3159, -7.0043, -6.9843, -7.0695, -7.0363, -6.9101, -6.9713,
        -6.8988, -7.0061, -6.9442, -7.0579], device='cuda:0',
       grad_fn=<SelectBackward0>)
Base model output: tensor([-6.9108,  3.0898, -6.7678,  2.9399, -6.8667,  4.4328, 14.5065, -2.9290,
         1.1001,  4.7732, -6.7857, -6.7669, -6.8650, -6.8332, -6.6798, -6.7445,
        -6.6867, -6.7851, -6.7233, -6.8461], device='cuda:0',
       grad_fn=<SelectBackward0>)
Are they close? False


In [ ]:
test = torch.tensor([[ 1,  5,  5,  1 , 5, 1,  4,  0]]).cuda()
ans = []
for i in range(10, 20):
    test[0, 0] = i
    gt = base_model.forward(test)
    ans.append(gt[0, -1])
ans = torch.stack(ans)
px.imshow(ans.detach().cpu(), color_continuous_midpoint=0, color_continuous_scale='RdBu')

trigrams =  [(10, 0, 1), (11, 0, 5), (12, 0, 5), (13, 0, 6), (14, 0, 9), (15, 0, 6), (16, 0, 8), (17, 0, 9), (18, 0, 3), (19, 0, 8)]


In [28]:
import einops
test = torch.tensor([[ 1,  5,  5,  1 , 5, 1,  4,  0]]).cuda()
ans = []
for i in range(10, 20):
    test[0, 0] = i
    x = base_model.token_embedding(test)
    query_acts = einops.einsum(x, model.components.attention.A, "batch seq d_model, d_model C -> batch seq C")
    key_acts = einops.einsum(x, model.components.attention.B, "batch seq d_model, C d_model -> batch seq C")
    component_acts = einops.einsum(query_acts, key_acts, "batch seqQ C, batch seqK C -> batch seqQ seqK C")
    ans.append(component_acts[0, 0, -1])
ans = torch.stack(ans)
px.imshow(ans.detach().cpu(), color_continuous_midpoint=0, color_continuous_scale='RdBu')



In [38]:
seq = [list(range(20))]
seq = torch.tensor(seq).cuda()
x = model.model.token_embedding(seq)
print(x.shape)
A = model.components.attention.A 
B = model.components.attention.B 
k_acts = torch.einsum('sd,cc->sc', x[0], B.cuda())    # [C]
q_acts = torch.einsum('sd,xc->sc', x[0], A.cuda())


torch.Size([1, 20, 16])


In [40]:
px.imshow(q_acts.detach().cpu(), color_continuous_midpoint=0, color_continuous_scale='RdBu')

In [22]:
import einops
x = base_model.token_embedding(test)
query_acts = einops.einsum(x, model.components.attention.A, "batch seq d_model, d_model C -> batch seq C")
key_acts = einops.einsum(x, model.components.attention.B, "batch seq d_model, C d_model -> batch seq C")
component_acts = einops.einsum(query_acts, key_acts, "batch seqQ C, batch seqK C -> batch seqQ seqK C")


scores = component_acts.sum(dim=-1) 

        # Apply causal mask if needed
scores = model.components.attention.causal_mask(scores)

        
patt = scores.softmax(dim=-1)
z = einops.einsum(patt, x, "batch seq1 seq2, batch seq2 d_model -> batch seq1 d_model")
out = einops.einsum(z, model.components.attention.ov, "batch seq d_model, d_model d_model -> batch seq d_model")
out = out + x
model.model.unembed(out)[0, -1]

tensor([-0.7388, -6.3002, -0.7098, -2.5913, -0.6578, -1.9118, -0.4276,  2.0457,
        12.9395,  1.5181, -0.6812, -0.7607, -0.6401, -0.6838, -0.7433, -0.5294,
        -0.7255, -0.6528, -0.7157, -0.8360], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [59]:
from spd.models.component_utils import (
    calc_causal_importances,
    calc_ci_l_zero,
    component_activation_statistics,
)

target_out, pre_weight_acts = model.forward_with_pre_forward_cache_hooks(
            test, module_names=list(components.keys())
        )
gates = {
        k.removeprefix("gates.").replace("-", "."): v for k, v in model.gates.items()
    }  # type: ignore
As = {module_name: components[module_name].A for module_name in components}
cis = calc_causal_importances(pre_weight_acts, As, gates)
px.imshow(cis[0]["attention"][0].detach().cpu(), color_continuous_midpoint=0, color_continuous_scale='RdBu')

In [14]:
out[0, -1]

tensor([ 2.0760, -2.0230, -1.2374, -1.2469, -1.0780, -0.9396,  1.4126, -1.4422,
         1.3203, -1.5197,  1.5964, -1.3095,  0.8329, -0.6197,  1.7893, -1.2619],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [19]:
base_model.forward(test)[0, -1]

tensor([-1.0258, -5.8353, -0.9948, -2.3342, -0.9520, -1.6445,  0.2631,  1.8429,
        12.3337,  1.7040, -0.9682, -1.0429, -0.9310, -0.9712, -1.0236, -0.8227,
        -1.0059, -0.9413, -0.9984, -1.1181], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [24]:
base_attn_out = base_model.attention(x)
base_attn_out = base_attn_out + x
base_model.unembed(base_attn_out)[0, -1]

tensor([-1.0258, -5.8353, -0.9948, -2.3342, -0.9520, -1.6445,  0.2631,  1.8429,
        12.3337,  1.7040, -0.9682, -1.0429, -0.9310, -0.9712, -1.0236, -0.8227,
        -1.0059, -0.9413, -0.9984, -1.1181], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [7]:
# Verify the actual reconstruction quality
original_weight = model.model.attention.weight.cuda()  # From the SPD model
reconstructed_weight = model.components['attention'].weight.cuda()  # A @ B

reconstruction_error = torch.norm(original_weight - reconstructed_weight)
print(f"Weight reconstruction error: {reconstruction_error.item():.2e}")

# If this is large (> 1e-3), then the decomposition didn't work well

Weight reconstruction error: 2.92e-04


In [15]:
# The SPD model should be identical to base model when using regular forward
spd_regular = model(test)  # Uses original weights
base_out = base_model(test)  # Original model

print(f"SPD regular: {spd_regular[0, -1]}")
print(f"Base model: {base_out[0, -1]}")
print(f"Are SPD and base identical? {torch.allclose(spd_regular, base_out, atol=1e-9)}")

# Then compare component reconstruction to SPD regular (not base_model)
component_out = model.forward_with_components(test, components=model.components)
print(f"Component reconstruction: {component_out[0, -1]}")
print(f"Are component and SPD regular close? {torch.allclose(component_out, spd_regular, atol=1e-3)}")

SPD regular: tensor([-1.1999, 12.4515, -1.2318,  1.7649, -1.2817,  0.4664,  1.9224,  2.4540,
        -5.4099, -1.6787, -1.3501, -1.1428, -1.1227, -1.1139, -1.1096, -1.1913,
        -1.1318, -1.3384, -1.2797, -1.2876], device='cuda:0',
       grad_fn=<SelectBackward0>)
Base model: tensor([-1.1999, 12.4515, -1.2318,  1.7649, -1.2817,  0.4664,  1.9224,  2.4540,
        -5.4099, -1.6787, -1.3501, -1.1428, -1.1227, -1.1139, -1.1096, -1.1913,
        -1.1318, -1.3384, -1.2797, -1.2876], device='cuda:0',
       grad_fn=<SelectBackward0>)
Are SPD and base identical? True
Component reconstruction: tensor([-5.1186,  6.3521, -5.0726,  2.4931, -5.1816,  2.9199, 10.2352, -0.7094,
        -1.2903,  3.2643, -5.1240, -5.0428, -5.0802, -5.0557, -4.9922, -5.0560,
        -4.9824, -5.1224, -5.0628, -5.1367], device='cuda:0',
       grad_fn=<SelectBackward0>)
Are component and SPD regular close? False


In [ ]:
import einops
component_reconstructed_weights = einops.einsum(model.components['attention'].A, model.components['attention'].B, 'dq c, c d -> dq d')
model_weight = model.components.attention.weight

print(f"Are SPD and base identical? {torch.allclose(component_reconstructed_weights, model_weight, atol=1e-9)}")

Are SPD and base identical? False


In [26]:
test = torch.tensor([[ 1,  5,  5,  15 , 5, 1,  4,  0]]).cuda()

# Get embeddings
embed = model.model.token_embedding(test)

# Method 1: Using stored model.components.attention.weight (A @ B)
scores1 = einops.einsum(embed, model.components.attention.weight, embed, 
                       "batch seq1 d_model, d_model d_model, batch seq2 d_model -> batch seq1 seq2")

# Method 2: Using original model weight
scores2 = einops.einsum(embed, model.model.attention.weight, embed, 
                       "batch seq1 d_model, d_model d_model, batch seq2 d_model -> batch seq1 seq2")

print(f"Attention scores identical: {torch.allclose(scores1, scores2, atol=1e-6)}")
print(f"Max difference: {(scores1 - scores2).abs().max().item():.2e}")

Attention scores identical: True
Max difference: 3.15e-05


In [29]:
import einops
original_weight = base_model.attention.weight
stored_component_weight = model.components.attention.weight
reconstructed_weight = einops.einsum(model.components['attention'].A, model.components['attention'].B, 'dq c, c d -> dq d')

print("=== WEIGHT MATRIX COMPARISON ===")
print(f"Original vs stored component: {torch.allclose(original_weight, stored_component_weight, atol=1e-4)}")
print(f"Original vs reconstructed A@B: {torch.allclose(original_weight, reconstructed_weight, atol=1e-6)}")
print(f"Stored component vs A@B: {torch.allclose(stored_component_weight, reconstructed_weight, atol=1e-9)}")

print(f"\nMax differences:")
print(f"Original vs stored component: {(original_weight - stored_component_weight).abs().max().item():.2e}")
print(f"Original vs reconstructed A@B: {(original_weight - reconstructed_weight).abs().max().item():.2e}")

=== WEIGHT MATRIX COMPARISON ===
Original vs stored component: False
Original vs reconstructed A@B: False
Stored component vs A@B: True

Max differences:
Original vs stored component: 1.35e-04
Original vs reconstructed A@B: 1.35e-04


In [1]:
original_weight.shape

NameError: name 'original_weight' is not defined

In [21]:
px.imshow(a.detach().cpu(), color_continuous_midpoint=0, color_continuous_scale='RdBu', labels={'x': 'Trigram', 'y': 'Component'}, title="Activation Scores for Each Trigram")

In [11]:
gt = base_model.forward(test)
gt[0, -1]

tensor([-6.9113,  3.0630, -6.7675,  2.9549, -6.8664,  4.4071, 14.5177, -2.9244,
         1.1120,  4.7802, -6.7846, -6.7666, -6.8647, -6.8338, -6.6794, -6.7444,
        -6.6864, -6.7845, -6.7234, -6.8459], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [30]:
import einops
toks = base_model.token_embedding(test).cuda()
scores = einops.einsum(toks, model.model.attention.weight, toks, 
                                "batch seq1 d_model, d_model d_model, batch seq2 d_model -> batch seq1 seq2")
            
            # Apply normalization
            
            # Apply causal mask if enabled

seq_len = toks.shape[1]
causal_mask = torch.triu(torch.ones(seq_len, seq_len, device=toks.device), diagonal=1).bool()
scores = scores.masked_fill(causal_mask, float('-inf'))
            
            # Apply softmax to get attention patterns
patterns = scores.softmax(dim=-1)
z = einops.einsum(patterns, toks, "batch seq1 seq2, batch seq2 d_model -> batch seq1 d_model")

In [31]:
px.imshow(patterns[0].detach().cpu(), color_continuous_midpoint=0, color_continuous_scale='RdBu')

In [32]:
query_acts = einops.einsum(toks, model.components.attention.A, "batch seq d_model, d_model C -> batch seq C")
key_acts = einops.einsum(toks, model.components.attention.B, "batch seq d_model, C d_model -> batch seq C")
component_acts = einops.einsum(query_acts, key_acts, "batch seqQ C, batch seqK C -> batch seqQ seqK C")
print(component_acts[0, -1, 0, :])

  
masked_component_acts = component_acts
        
        # Sum over components to get final attention scores
scores = masked_component_acts.sum(dim=-1)
     

scores = model.components.attention.causal_mask(scores)
  
recon_patt = scores.softmax(dim=-1)

recon_z = einops.einsum(recon_patt, toks, "batch seq1 seq2, batch seq2 d_model -> batch seq1 d_model")

tensor([-1.5468e+01, -4.7089e+01, -8.4184e-01, -1.2774e+00,  3.9521e+01,
         4.2989e+01, -8.2040e+01,  4.9644e+01,  3.8868e+01, -3.0488e+00,
         4.2796e-02, -2.9017e+01,  1.3020e+00, -8.9676e-01,  9.1920e-01,
        -5.4216e+00], device='cuda:0', grad_fn=<SliceBackward0>)


In [11]:
import einops

In [67]:
attn_out = einops.einsum(recon_z, base_model.attention.OV, "batch seq d_model, d_model d_model -> batch seq d_model") + toks
unembed = model.model.unembed(attn_out)
unembed[0, -1]

tensor([-7.1350,  0.9719, -6.9586,  4.4579, -7.0308, 16.8835,  4.9442, -0.7443,
         0.0268,  5.8192, -7.1231, -6.9750, -7.1507, -7.0017, -7.0966, -7.0753,
        -6.9474, -7.0473, -6.9358, -6.8209], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [69]:
model.forward_with_components(test.cuda(), components = components)[0, -1]

tensor([-7.0484,  2.6364, -6.9360,  3.3455, -7.0532,  8.0820, 11.3513, -1.7737,
         0.4933,  5.7049, -7.0002, -6.9403, -7.0398, -6.9713, -6.9375, -6.9688,
        -6.8738, -6.9789, -6.9028, -6.9399], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [33]:
px.imshow(recon_patt[0].detach().cpu(), color_continuous_midpoint=0, color_continuous_scale='RdBu')

In [13]:
# Compare intermediate outputs step by step
test_input = torch.tensor([[ 1,  5,  5,  17 , 5, 1,  4,  0]]).cuda()

print("=== STEP BY STEP COMPARISON ===")

# Step 1: Token embeddings
base_embeds = base_model.token_embedding(test_input)
spd_embeds = model.model.token_embedding(test_input)
print(f"Token embeddings identical: {torch.allclose(base_embeds, spd_embeds, atol=1e-9)}")

# Step 2: Attention patterns (you already verified these are identical)
# Step 3: Attention output 
base_attn_out = base_model.attention(base_embeds)
spd_attn_out = model.model.attention(spd_embeds)
print(f"Attention outputs identical: {torch.allclose(base_attn_out, spd_attn_out, atol=1e-9)}")

# Step 4: Final logits
base_logits = base_model.unembed(base_attn_out)
spd_logits = model.model.unembed(spd_attn_out)
component_logits = model.forward_with_components(test_input.cuda(), components=model.components)[0, -1]
print(f"Final logits identical: {torch.allclose(base_logits, spd_logits, atol=1e-9)}")
print(f"Component logits identical: {torch.allclose(component_logits, spd_logits, atol=1e-9)}")
# Check if unembed weights are the same
print(f"Unembed weights identical: {torch.allclose(base_model.unembed.weight, model.model.unembed.weight, atol=1e-6)}")

=== STEP BY STEP COMPARISON ===
Token embeddings identical: True
Attention outputs identical: True
Final logits identical: True
Component logits identical: False
Unembed weights identical: True


In [31]:
base_logits[0, -1]

tensor([-7.2862, -0.2895, -7.4335, -2.4450, -7.6027,  5.8455,  5.5398,  4.9212,
         4.0170, 16.0107, -7.5173, -7.5031, -7.6434, -7.5300, -7.6050, -7.5784,
        -7.2901, -7.5892, -7.6428, -7.4486], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [32]:
spd_logits[0, -1]

tensor([-7.2862, -0.2895, -7.4335, -2.4450, -7.6027,  5.8455,  5.5398,  4.9212,
         4.0170, 16.0107, -7.5173, -7.5031, -7.6434, -7.5300, -7.6050, -7.5784,
        -7.2901, -7.5892, -7.6428, -7.4486], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [42]:
from spd.utils import calc_kl_divergence_lm
calc_kl_divergence_lm(base_logits, spd_logits)

tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)

In [45]:
log_q = torch.log_softmax(spd_logits, dim=-1)
p = torch.softmax(base_logits, dim=-1)

In [53]:
calc_kl_divergence_lm(log_q[0, -1], p[0, -1], pre_softmax=False)

tensor(14.7413, device='cuda:0', grad_fn=<MeanBackward0>)

In [56]:
import torch.nn.functional as F
attn = True
pred = base_logits
target = spd_logits
if attn:
        pred = pred.view(-1, pred.size(-1))
        target = target.view(-1, target.size(-1))
    
log_q = torch.log_softmax(pred, dim=-1)  # log Q
p = torch.softmax(target, dim=-1)  # P
kl = F.kl_div(log_q, p, reduction="none")  # P · (log P − log Q)

torch.Size([20, 16])

In [ ]:
test = torch.tensor([[ 1,  5,  5,  10 , 5, 1,  4,  0]]).cuda()

# 1. Base model forward
base_out = base_model.forward(test)

# 2. SPD model regular forward (should be identical to base)
spd_out = model(test)  # Uses original weights

# 3. SPD model with components (uses A @ B decomposition)
components = {k.removeprefix("components.").replace("-", "."): v.cuda() for k, v in model.components.items()}
component_out = model.forward_with_components(test, components=components)

print("Comparison of final logits:")
print(f"Base model:        {base_out[0, -1]}")
print(f"SPD regular:       {spd_out[0, -1]}")  
print(f"SPD components:    {component_out[0, -1]}")

print(f"\nAre base and SPD regular identical? {torch.allclose(base_out, spd_out, atol=1e-6)}")
print(f"Are SPD regular and components identical? {torch.allclose(spd_out, component_out, atol=1e-6)}")

Comparison of final logits:
Base model:        tensor([-1.1999, 12.4515, -1.2318,  1.7649, -1.2817,  0.4664,  1.9224,  2.4540,
        -5.4099, -1.6787, -1.3501, -1.1428, -1.1227, -1.1139, -1.1096, -1.1913,
        -1.1318, -1.3384, -1.2797, -1.2876], device='cuda:0',
       grad_fn=<SelectBackward0>)
SPD regular:       tensor([-1.1999, 12.4515, -1.2318,  1.7649, -1.2817,  0.4664,  1.9224,  2.4540,
        -5.4099, -1.6787, -1.3501, -1.1428, -1.1227, -1.1139, -1.1096, -1.1913,
        -1.1318, -1.3384, -1.2797, -1.2876], device='cuda:0',
       grad_fn=<SelectBackward0>)
SPD components:    tensor([-5.1217,  6.3578, -5.0756,  2.4950, -5.1846,  2.9255, 10.2382, -0.7096,
        -1.2910,  3.2660, -5.1271, -5.0458, -5.0832, -5.0587, -4.9952, -5.0590,
        -4.9853, -5.1255, -5.0658, -5.1396], device='cuda:0',
       grad_fn=<SelectBackward0>)

Are base and SPD regular identical? True
Are SPD regular and components identical? False


In [ ]:
#Compare component recon with component model
embed = model.model.token_embedding(test)
print(test.shape, model.components.attention.weight.shape)
scores = einops.einsum(embed, model.components.attention.weight, embed, 
                                "batch seq1 d_model, d_model d_model, batch seq2 d_model -> batch seq1 seq2")
            
seq_len = embed.shape[1]
causal_mask = torch.triu(torch.ones(seq_len, seq_len, device='cuda'), diagonal=1).bool()
scores = scores.masked_fill(causal_mask, float('-inf'))
            
patterns = scores.softmax(dim=-1)
print(patterns)
z = einops.einsum(patterns, embed, "batch seq1 seq2, batch seq2 d_model -> batch seq1 d_model")
gt_out = einops.einsum(z, model.components.attention.ov, "batch seq d_model, d_model d_model -> batch seq d_model")
print(gt_out[0, -1])

torch.Size([1, 8]) torch.Size([16, 16])
tensor([[[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [4.8180e-01, 5.1820e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [3.1735e-01, 3.4132e-01, 3.4132e-01, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [3.6259e-01, 3.1870e-01, 3.1870e-01, 1.1589e-10, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [1.3405e-11, 1.4418e-11, 1.4418e-11, 1.0000e+00, 1.4418e-11,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [1.0910e-11, 1.1783e-11, 1.1783e-11, 1.0000e+00, 1.1783e-11,
          1.0910e-11, 0.0000e+00, 0.0000e+00],
         [1.1048e-11, 1.1931e-11, 1.1931e-11, 1.0000e+00, 1.1931e-11,
          1.1048e-11, 1.0666e-11, 0.0000e+00],
         [7.5299e-09, 8.0462e-09, 8.0462e-09, 1.0000e+00, 8.0462e-09,
          7.5299e-09, 7.4094e-09, 1.6845e-07]]], device='cuda:0',
     

torch.Size([16, 16])

In [59]:
model.components.attention.weight == recon_qk

tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,

In [16]:
query_acts = einops.einsum(embed, model.components.attention.A, "batch seq d_model, d_model C -> batch seq C")
key_acts = einops.einsum(embed, model.components.attention.B, "batch seq d_model, C d_model -> batch seq C")
component_acts = einops.einsum(query_acts, key_acts, "batch seqQ C, batch seqK C -> batch seqQ seqK C")
  
 
        # Sum over components to get final attention scores
scores = component_acts.sum(dim=-1)  # [batch, seqQ, seqK]
        

scores = model.components.attention.causal_mask(scores)
        

patt = scores.softmax(dim=-1)
z = einops.einsum(patt, embed, "batch seq1 seq2, batch seq2 d_model -> batch seq1 d_model")
out = einops.einsum(z, model.components.attention.ov, "batch seq d_model, d_model d_model -> batch seq d_model")
print(out[0, -1])

tensor([-3.2958, -2.3729, -0.7737,  2.4028,  0.3793, -0.2801, -2.0956,  1.0342,
        -1.2233, -1.2937,  2.3865,  0.2426,  0.3037, -0.1614, -2.0324, -0.5992],
       device='cuda:0', grad_fn=<SelectBackward0>)


In [22]:
recon_qk = einops.einsum(model.components.attention.A, model.components.attention.B, "d_modelQ C, C d_model -> d_modelQ d_model")
qk = base_model.attention.weight
print(torch.allclose(recon_qk, qk, atol=1e-6))

False


In [33]:
((recon_qk - qk)**2).mean()

tensor(0.0010, device='cuda:0', grad_fn=<MeanBackward0>)

In [31]:
from spd.losses import calc_faithfulness_loss
calc_faithfulness_loss(components, base_model, 16**2, device='cuda')

tensor(0.0010, device='cuda:0', grad_fn=<DivBackward0>)